<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/lessons3/Embedding_word2vec_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding word2vec fasttext

In [21]:
!pip install annoy
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Импортируем библиотеки

In [22]:
import re
import pandas as pd
import numpy as np
import annoy

import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from string import punctuation
from gensim.models import Word2Vec, FastText

for i in ['stopwords', 'punkt']:
  nltk.download(i)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Подготавливаем данные

In [23]:
#@title Скачиваем данные
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-13 13:50:55--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-13 13:50:55--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50b7d13307a39780c6e97af7ec.dl.dropboxusercontent.com/cd/0/inline/BnILbZ9jA-KJDy-bcVuYpXmCoM3OPll-TN6Qml8_8QIpyITvvjNRGPM2mTRor3W6abARiaNWdvZHXlJhU2lY6VfcA-EJtW0RImY6NWEuPYrpzfChrMwk-uPMAg-zBtxMbxz882GIiWwHlQY6IaCuXRPgSwtxkfD_rNFvufQT1AsYrw/file# [following]
--2022-06-13 13:50:56--  https://uc50b7d13307a39780c6e97af7ec.dl.dropboxusercontent.com/cd/0/inline/BnILbZ9jA-KJDy-bcVuYpXmCoM3OPll-TN6Qml8_8QIpyITvvjNRGPM2mTRor3W6abAR

## 1. Объеденяем в одну выборку


In [24]:
pos = pd.read_csv('positive.csv', sep=";",  usecols=[3], names=['tweet'])
pos['label'] = ['positive'] * len(pos)
neg = pd.read_csv('negative.csv', sep=";",  usecols=[3], names=['tweet'])
neg['label'] = ['negative'] * len(neg)
df = pos.append(neg)
df = df.reset_index().drop('index', axis=1)
watch_df = df.iloc[np.r_[0:5, -5:0]]
print(watch_df)

                                                    tweet     label
0       @first_timee хоть я и школота, но поверь, у на...  positive
1       Да, все-таки он немного похож на него. Но мой ...  positive
2       RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3       RT @digger2912: "Кто то в углу сидит и погибае...  positive
4       @irina_dyshkant Вот что значит страшилка :D\nН...  positive
226829  Но не каждый хочет что то исправлять:( http://...  negative
226830  скучаю так :-( только @taaannyaaa вправляет мо...  negative
226831          Вот и в школу, в говно это идти уже надо(  negative
226832  RT @_Them__: @LisaBeroud Тауриэль, не грусти :...  negative
226833  Такси везет меня на работу. Раздумываю приплат...  negative


## 2. На основе word2vec/fasttext/glove/слоя Embedding реализуем метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)


In [25]:
emoji_list = [":)",":‑)",":-]",":-3",":->","8-)",":-}",":o)",":c)",":^)","=]","=)","<3",":-(",":(",":c",":<",":[",">:[",":{",">:(",":-c",":-< ",":-[",":-||"]
morph = MorphAnalyzer()

# Стоп слова нужны на русском
stop_words = set(stopwords.words("russian"))

In [26]:
def del_emoji(words):
  '''
    Функция для удаления текстовых эмоций
  '''
  for el in emoji_list:
    words = words.replace(el, '')
  return words

def preprocessing_text(txt: str):
  '''
    Предобработка текста, т.е. удаляем лишнее, заменяем и т.д.
  '''
  # Изначально все в нижний регистр переводим
  txt = txt.lower()
  # Удаляем всех пользователей: @user
  txt = re.sub(r"@[\w]*", '', txt)
  # Удаляем эмодзи, чтобы не терялась смысловая нагрузка надо бы их конечно перобразовать в слова, на подобии: :) -> "счастлив" или :-( -> "печально", но мы это опустим
  txt = del_emoji(txt)
  # Удаляем всю пунктуацию
  txt = re.sub(r'[^\w\s]', ' ', txt)
  # Заменим спец. символы c цифрами на пробелы
  txt = re.sub(r'[^a-zA-Zа-яёA-ЯЁ]', ' ', txt)
  # Удалим из текста слова длиной менее 2-х символов
  txt = [w for w in txt.split() if len(w) > 2]
  # Удалим стоп-слова
  txt = [w for w in txt if w not in stop_words] 
  # Приведем к нормальной форме
  return [morph.parse(w)[0].normal_form for w in txt]  

def get_response(question, index, model, index_map):
  '''
    Получение ответа из модели
  '''
  question = preprocessing_text(question)
  vector = np.zeros(300)
  norm = 0
  for word in question:
      if word in model.wv:
          vector += model.wv[word]
          norm += 1
  if norm > 0:
      vector = vector / norm
  answers = index.get_nns_by_vector(vector, 5, )
  return [index_map[i] for i in answers]

In [27]:
df['preprocessing'] = df.tweet.apply(preprocessing_text)
df.reset_index(drop=True)
df.head()

,tweet,label,preprocessing
0,"@first_timee хоть я и школота, но поверь, у на...",positive,"[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...",positive,"[таки, немного, похожий, мальчик, равно]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,"[идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,"[угол, сидеть, погибать, голод, ещё, порция, в..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,"[значит, страшилка, блин, посмотреть, часть, с..."


In [29]:
# Для min_count все же используем не 1, а 5 (это значение по умолчанию), т.е. должно встретиться минимум 5 раз
# Модель Word2Vec
modelW2V = Word2Vec(sentences=df['preprocessing'].values, size=300, window=5, min_count=5, workers=8)

# Модель FastText
modelFT = FastText(sentences=df['preprocessing'].values, size=300, min_count=5, window=5, workers=8)

In [30]:
w2v_index = annoy.AnnoyIndex(300, 'angular')
ft_index = annoy.AnnoyIndex(300, 'angular')

counter = 0
index_map = {}

for line in df['tweet']:
    n_w2v, n_ft = 0, 0

    index_map[counter] = line
    tweet = preprocessing_text(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in tweet:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)


True

## 3. Проверить насколько хорошо работают подходы

In [31]:
quest_tweet = "В школе хорошо"

In [32]:
#@title Проверка у Word2Vec с позитивным комментом
get_response(quest_tweet, w2v_index, modelW2V, index_map)

['@windrty @LudmilaSadovaja @volodya_Kharkov @zero_home секспросвет в школе, гендерство ) еще?',
 '@__wishmaster__ в 12:15 на кванте\nМне ж еше в школу надо :D',
 'Ну ничего себе, теперь и у 21 школы #подслушано есть)',
 'RT @lesja_po_878104: наконец-то школа избавилась от меня :D',
 'Нашаа школа обложалась крч. Но 18 школа крутяк выступила :)']

In [33]:
#@title Проверка у FastText с позитивным комментом
get_response(quest_tweet, ft_index, modelFT, index_map)

['не в школе, так хоть в одноклассниках я отличница ;)',
 'RT @Hell_Thing: @ludiPrijatelj я в школе, какая постель? :D',
 'было весело сегодня в школе, когда вместо звонка на урок, включили AC/DC - Highway to hell на всю школу :D',
 'RT @id132932855: В пятницу в школе будет батл :3\n Между учениками и учителями, уахах:8\n Будет весело :D',
 '@NastyaNaaastya о, еще и в школе другой.у нас просто комиссии были.но в др школах мб даже помягче с дисциплиной будет)']

In [34]:
quest_tweet = "Плохое настроение"

In [35]:
#@title Проверка у Word2Vec с негативным комментом
get_response(quest_tweet, w2v_index, modelW2V, index_map)

['@NataliaShavel @sergeylazarev мне она всегда настроение поднимает.)))',
 '@borisov_vasiliy @NikolinaTheOne , Вася, твить больше %) поднимай настроение %)',
 'Шавкун,ты всегда поднимешь настроение:) какой бы тяжёлай..ситуация не была :)\n#певцыотбога',
 '@Verrraverrra Приветик, Верочка ))@wmk2u @LanaFom @akutata11 @ZhSlava @alex07s @Weelelee @Vikiiri Веселого настроения ))',
 'Хорошее настроение *-*\nНесколько слов, а настроение лучше сразу\nСпасиииибо:**']

In [36]:
#@title Проверка у FastText с негативным комментом
get_response(quest_tweet, ft_index, modelFT, index_map)

['@because_malik Та у меня вроде как настроение хорошее )',
 '@cuddlewthzourry у меня сегодня плохое настроение ссорьки )',
 'Хорошее настроение *-*\nНесколько слов, а настроение лучше сразу\nСпасиииибо:**',
 '-Хорошее настроение у меня)\nЫчкынале!:D @Alinkaaaaa98',
 '@Valeriya_Zarya @viksimon1 прекрасного настроения вам)']

## Вывод
Судя по резултату... Если с положительным комментарием еще можно как-то связать, то контекст у негативного комментария далеко утерян. Может быть это связано с тем что потерли смайлики или перебрали с предобработкой, надо будет попробовать.